In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [3]:
IMG_SIZE = 64

def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image / 255.0  # Normalize to [0, 1]
    return image

<img src="old/plate/blur/0_aug_1.jpg" width=400>

In [12]:
load_model = tf.keras.models.load_model('model.h5')

new_image = load_and_preprocess_image('old/plate/blur/0_aug_1.jpg')
new_image = np.expand_dims(new_image, axis=0)
prediction = load_model.predict(new_image)

if prediction > 0.5:
    print('Not blurry')
else:
    print('Blurry')

prediction

1/1 [==============================] - 0s 202ms/step
Blurry


array([[0.00561011]], dtype=float32)

<img src="old/plate/not_blur/0.jpg" width=400>

In [13]:
new_image = load_and_preprocess_image('old/plate/not_blur/0.jpg')
new_image = np.expand_dims(new_image, axis=0)
prediction = load_model.predict(new_image)

if prediction > 0.5:
    print('Not blurry')
else:
    print('Blurry')

prediction

1/1 [==============================] - 0s 46ms/step
Not blurry


array([[0.57056665]], dtype=float32)